In [6]:
from openai import OpenAI
import base64
import requests
import json
from functions import convert_pdfs_to_pngs, get_the_files_with_paths
import os
from dotenv import load_dotenv

In [7]:
# Załaduj zmienne środowiskowe z pliku .env
load_dotenv()

# przypisz zmienne środowiskowe do zmiennych
api_key = os.getenv('OPENAI_API')
pdf_path = os.getenv('PDF_DIRECTORY')
png_path = os.getenv('OUTPUT_DIRECTORY')

In [8]:
# Konwertowanie PDF'ów na PNG z folderu PDFs do folderu PNGs
convert_pdfs_to_pngs(pdf_path, png_path)

# Wybieranie plików do sprawdzenia
files_to_analyze = get_the_files_with_paths(png_path)

In [9]:
# Jeden pliczek do testów
file  = files_to_analyze[3]
file

'/Users/kamilczaja/Library/Mobile Documents/com~apple~CloudDocs/Documents/3. Resources/Paragony/PNGs/Scanned Document 2_page_3.png'

In [10]:
# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

In [11]:
# Getting the base64 string
base64_image = encode_image(file)

In [12]:
prompt = """
Jesteś analizazatorem paragonów. Na podstawie podanego zdjęcia paragonu, wyciągnij z niego jak najwiecej informacji.
Korzystaj z polskich znaków, tam gdzie jest to wymagane.
Odpowiadaj tylko i wyłącznie w postaci JSONA, korzystając z ponizszego szablonu:
{
  "transactions": [
    {
      "id": "unique_transaction_id_1",
      "receipt_number": "1234567890",
      "timestamp": "2023-09-23T14:55:00Z",
      "date": "2023-09-23",
      "store": {
        "name": "Dealz",
        "address": "Ul. Dąbrowskiego 79a, Poznań",
        "additional_information": "Sklep: P2339",
        "NIP": "7811963267"
      },
      "cashier": "John Doe",
      "terminal_id": "POS12345",
      "total": 53.00,
      "currency": "PLN",
      "payment_method": "credit_card",
      "taxes": [
        {
          "type": "VAT",
          "rate": 0.23,
          "amount": 8.23
        }
      ],
      "discounts": [
        {
          "description": "Weekend discount",
          "amount": 5.00
        }
      ],
      "items": [
        {
          "name": "CHEESE",
          "unit_price": {
            "before_discount": 10.00,
            "after_discount": 9.50
          },
          "quantity": 1,
          "total_price": {
            "before_discount": 10.00,
            "after_discount": 9.50
          },
          "category": "Dairy",
          "tax_rate": 0.23,
          "discount": {
            "description": "Cheese discount",
            "amount": 0.50
          }
        },
        {
          "name": "Napo",
          "unit_price": {
            "before_discount": 2.00,
            "after_discount": 2.00
          },
          "quantity": 1,
          "total_price": {
            "before_discount": 2.00,
            "after_discount": 2.00
          },
          "category": "Beverage",
          "tax_rate": 0.23,
          "discount": null
        }
      ],
      "comments": "Sale on beverages",
      "category": "Groceries"
    }
  ]
}
"""

In [13]:
headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {api_key}"
}

payload = {
  "model": "gpt-4o",
  "messages": [
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": f"{prompt}"
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}"
          }
        }
      ]
    }
  ]
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

print(response.json())

{'id': 'chatcmpl-9aPAlkIHT58AzhO0dC7MpWuT2de1m', 'object': 'chat.completion', 'created': 1718463599, 'model': 'gpt-4o-2024-05-13', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '```json\n{\n  "transactions": [\n    {\n      "id": "F518755",\n      "receipt_number": "W787066",\n      "timestamp": "2024-01-10T01:08:00Z",\n      "date": "2024-01-10",\n      "store": {\n        "name": "ORLEN S.A.",\n        "address": "Ul. Chemików 7, 09-411 Płock",\n        "additional_information": "Stacja Paliw nr 1317 w Zakopanem, ul. Ustup 1B, 34-500 Zakopane",\n        "NIP": "774-00-01-454"\n      },\n      "cashier": "130",\n      "terminal_id": "K4",\n      "total": 257.16,\n      "currency": "PLN",\n      "payment_method": "cash",\n      "taxes": [\n        {\n          "type": "VAT",\n          "rate": 0.23,\n          "amount": 48.09\n        }\n      ],\n      "discounts": [],\n      "items": [\n        {\n          "name": "Effecta Diesel CN27102011 (AD(3)",\n         

In [15]:
content = response.json()['choices'][0]['message']['content']
content

'```json\n{\n  "transactions": [\n    {\n      "id": "F518755",\n      "receipt_number": "W787066",\n      "timestamp": "2024-01-10T01:08:00Z",\n      "date": "2024-01-10",\n      "store": {\n        "name": "ORLEN S.A.",\n        "address": "Ul. Chemików 7, 09-411 Płock",\n        "additional_information": "Stacja Paliw nr 1317 w Zakopanem, ul. Ustup 1B, 34-500 Zakopane",\n        "NIP": "774-00-01-454"\n      },\n      "cashier": "130",\n      "terminal_id": "K4",\n      "total": 257.16,\n      "currency": "PLN",\n      "payment_method": "cash",\n      "taxes": [\n        {\n          "type": "VAT",\n          "rate": 0.23,\n          "amount": 48.09\n        }\n      ],\n      "discounts": [],\n      "items": [\n        {\n          "name": "Effecta Diesel CN27102011 (AD(3)",\n          "unit_price": {\n            "before_discount": 39.87,\n            "after_discount": 39.87\n          },\n          "quantity": 6.45,\n          "total_price": {\n            "before_discount": 257.

In [16]:
# Remove the ```json and ``` markers
cleaned_content = content.strip('```json\n').strip('\n```')

# Parse the cleaned string as JSON
json_data = json.loads(cleaned_content)

# Print the parsed JSON object to verify
print(json.dumps(json_data, indent=4))

{
    "transactions": [
        {
            "id": "F518755",
            "receipt_number": "W787066",
            "timestamp": "2024-01-10T01:08:00Z",
            "date": "2024-01-10",
            "store": {
                "name": "ORLEN S.A.",
                "address": "Ul. Chemik\u00f3w 7, 09-411 P\u0142ock",
                "additional_information": "Stacja Paliw nr 1317 w Zakopanem, ul. Ustup 1B, 34-500 Zakopane",
                "NIP": "774-00-01-454"
            },
            "cashier": "130",
            "terminal_id": "K4",
            "total": 257.16,
            "currency": "PLN",
            "payment_method": "cash",
            "taxes": [
                {
                    "type": "VAT",
                    "rate": 0.23,
                    "amount": 48.09
                }
            ],
            "discounts": [],
            "items": [
                {
                    "name": "Effecta Diesel CN27102011 (AD(3)",
                    "unit_price": {
          